# Custom Exceptions - Coding

In [1]:
class TimeoutError(Exception):
    """Timeout exception"""

In [2]:
try:
    raise TimeoutError('timeout occured')
except TimeoutError as ex:
    print(repr(ex))

TimeoutError('timeout occured')


In [4]:
import sys

try:
    raise TimeoutError('timeout occured')
except:
    ex_type, ex, tb = sys.exc_info()
    print(ex_type)
    print(ex)
    print(tb)

<class '__main__.TimeoutError'>
timeout occured


In [5]:
import sys

try:
    raise TimeoutError('timeout occured')
except Exception as ex:
    print(ex.args, ex.__traceback__)


('timeout occured',) <traceback object at 0x7f7f082db040>


In [6]:
class ReadOnlyError(AttributeError):
    """Indicates an attribute is read-only"""

In [7]:
try:
    raise ReadOnlyError('Account number is read-only', 'BA10001')
except ReadOnlyError as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-only', 'BA10001')


In [8]:
try:
    raise ReadOnlyError('Account number is read-only', 'BA10001')
except AttributeError as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-only', 'BA10001')


In [9]:
class WebScraperException(Exception):
    """Base exception for WebScraper"""
    
class HTTPException(WebScraperException):
    """General HTTP exception for WS"""
    
class InvalidUrlException(HTTPException):
    """Indicates the url is invalid (dns lookup fail)"""
    
class TimeoutException(HTTPException):
    """Indicates a general timeout exception in http connectivity"""
    
class PingTimeoutException(TimeoutException):
    """Ping time out"""

class LoadTimeoutException(TimeoutException):
    """Page load time out"""
    
class ParserException(WebScraperException):
    """General page parsing exception"""

In [10]:
try:
    raise PingTimeoutException("Ping to www... time out")
except WebScraperException as ex:
    print(repr(ex))

PingTimeoutException('Ping to www... time out')


In [13]:
class APIException(Exception):
    """Base API exception"""

In [15]:
class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type errors"""
    
class DBException(ApplicationException):
    """General database exception"""
    
class DBConnectionError(DBException):
    """Indicates an error connecting to database"""
    
class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""
    
class NotFoundError(ClientException):
    """Indicates resource was not found"""

class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""
    


In [16]:
class Account:
    def __init__(self, account_id, account_type):
        self.account_id = account_id
        self.account_type = account_type

In [17]:
def lookup_account_by_id(account_id):
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(f"Account number {account_id} is invalid.")
    
    if account_id < 100:
        raise DBConnectionError("Permanent failure connecting to database.")
    elif account_id < 200:
        raise NotAuthorizedError('User does not have permissions to read this account.')
    elif account_id < 300:
        raise NotFoundError('Account not found')
    else:
        return Account(account_id, 'Savings')

In [18]:
from http import HTTPStatus

In [19]:
def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except ApplicationException as ex:
        return HTTPStatus.INTERNAL_SERVER_ERROR, str(ex)
    except NotFoundError as ex:
        return HTTPStatus.NOT_FOUND, f"The account {account_id} does not exist"
    except NotAuthorizedError as ex:
        return HTTPStatus.UNAUTHORIZED, f"You do not have the proper authorization"
    except ClientException as ex:
        return HTTPStatus.BAD_REQUEST,  str(ex)
    else:
        return HTTPStatus.OK, {"id": account.account_id, "type": account.account_type}

In [20]:
get_account('abc')

(<HTTPStatus.BAD_REQUEST: 400>, 'Account number abc is invalid.')

In [21]:
get_account(50)

(<HTTPStatus.INTERNAL_SERVER_ERROR: 500>,
 'Permanent failure connecting to database.')

In [22]:
get_account(350)

(<HTTPStatus.OK: 200>, {'id': 350, 'type': 'Savings'})

In [23]:
class APIException(Exception):
    """Base API exception"""
    
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "API exception occured."
    user_err_msg = 'We are sorry! An unexpected error occurred on our end.'
    
    def __init__(self, *args, user_err_msg=None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)
            
        if user_err_msg is not None:
            self.user_err_msg = user_err_msg
            

In [26]:
try:
    raise APIException('custom message...', 10, 20, user_err_msg="custom user msg")
except APIException as ex:
    print(repr(ex))
    print(ex.user_err_msg)

APIException('custom message...', 10, 20)
custom user msg


In [27]:
import json

In [28]:
class APIException(Exception):
    """Base API exception"""
    
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "API exception occured."
    user_err_msg = 'We are sorry! An unexpected error occurred on our end.'
    
    def __init__(self, *args, user_err_msg=None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)
            
        if user_err_msg is not None:
            self.user_err_msg = user_err_msg
            
    def to_json(self):
        err_object = {"status": self.http_status, 
                      "message": self.user_err_msg}
        return json.dumps(err_object)

In [29]:
try:
    raise APIException()
except APIException as ex:
    print(repr(ex))
    print(ex.to_json())

APIException('API exception occured.')
{"status": 500, "message": "We are sorry! An unexpected error occurred on our end."}


In [34]:
from datetime import datetime

In [44]:
class APIException(Exception):
    """Base API exception"""
    
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "API exception occured."
    user_err_msg = 'We are sorry! An unexpected error occurred on our end.'
    
    def __init__(self, *args, user_err_msg=None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)
            
        if user_err_msg is not None:
            self.user_err_msg = user_err_msg
            
    def to_json(self):
        err_object = {"status": self.http_status, 
                      "message": self.user_err_msg}
        return json.dumps(err_object)
    
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "http_status": self.http_status,
            "message": self.args[0] if self.args else self.internal_err_msg,
            "args": self.args[1:]
        }
        print(f"EXCEPTION: {datetime.utcnow().isoformat()}: {exception}")

In [45]:
try:
    raise APIException()
except APIException as ex:
    ex.log_exception()
    print()
    print(ex.to_json())

EXCEPTION: 2023-07-02T12:46:03.781072: {'type': 'APIException', 'http_status': <HTTPStatus.INTERNAL_SERVER_ERROR: 500>, 'message': 'API exception occured.', 'args': ()}

{"status": 500, "message": "We are sorry! An unexpected error occurred on our end."}


In [46]:
class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type errors"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "Generic server side exception"
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
    
class DBException(ApplicationException):
    """General database exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "Database exception"
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
        
class DBConnectionError(DBException):
    """Indicates an error connecting to database"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "Database connection exception"
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
        
class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""
    http_status = HTTPStatus.BAD_REQUEST
    internal_err_msg = "Client submitted bad request."
    user_err_msg = "A bad request was received."
        
        
class NotFoundError(ClientException):
    """Indicates resource was not found"""
    http_status = HTTPStatus.NOT_FOUND
    internal_err_msg = "Resource not found."
    user_err_msg = "Requested resource was not found."
    
    
class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""
    http_status = HTTPStatus.UNAUTHORIZED
    internal_err_msg = "Client not authorized to perform operation."
    user_err_msg = "You are not authorized to perform this request."

In [47]:
def lookup_account_by_id(account_id):
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(f"Account number {account_id} is invalid.",
                             f"account_id= {account_id}", 
                             f"type error - account number not an integer")
    
    if account_id < 100:
        raise DBConnectionError("Permanent failure connecting to database.","db=db01")
    elif account_id < 200:
        raise NotAuthorizedError('User does not have permissions to read this account.',
                                f"account_id={account_id}")
    elif account_id < 300:
        raise NotFoundError('Account not found',f"account_id={account_id}")
    else:
        return Account(account_id, 'Savings')

In [48]:
def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except APIException as ex:
        ex.log_exception()
        return ex.to_json()
    else:
        return HTTPStatus.OK, {"id": account.account_id, "type": account.account_type}

In [49]:
get_account("ABC")

EXCEPTION: 2023-07-02T12:46:04.586142: {'type': 'ClientException', 'http_status': <HTTPStatus.BAD_REQUEST: 400>, 'message': 'Account number ABC is invalid.', 'args': ('account_id= ABC', 'type error - account number not an integer')}


'{"status": 400, "message": "A bad request was received."}'

In [50]:
get_account(50)

EXCEPTION: 2023-07-02T12:46:34.441089: {'type': 'DBConnectionError', 'http_status': <HTTPStatus.INTERNAL_SERVER_ERROR: 500>, 'message': 'Permanent failure connecting to database.', 'args': ('db=db01',)}


'{"status": 500, "message": "We are sorry. An unexpected error occurred on our end."}'

In [51]:
get_account(150)

EXCEPTION: 2023-07-02T12:46:55.292003: {'type': 'NotAuthorizedError', 'http_status': <HTTPStatus.UNAUTHORIZED: 401>, 'message': 'User does not have permissions to read this account.', 'args': ('account_id=150',)}


'{"status": 401, "message": "You are not authorized to perform this request."}'

In [52]:
get_account(250)

EXCEPTION: 2023-07-02T12:47:08.340102: {'type': 'NotFoundError', 'http_status': <HTTPStatus.NOT_FOUND: 404>, 'message': 'Account not found', 'args': ('account_id=250',)}


'{"status": 404, "message": "Requested resource was not found."}'

In [53]:
get_account(350)

(<HTTPStatus.OK: 200>, {'id': 350, 'type': 'Savings'})

In [64]:
class AppException(Exception):
    """generic application exception"""
    
class NegativeIntegerError(AppException, ValueError):
    """Used to indicate an error when an integer is negative"""

In [67]:
ex = NegativeIntegerError()
isinstance(ex, AppException), isinstance(ex, ValueError)

(True, True)

In [65]:
def set_age(age):
    if age < 0:
        raise NegativeIntegerError("age cannot be negative")

In [66]:
try:
    set_age(-10)
except ValueError as ex:
    print(repr(ex))

NegativeIntegerError('age cannot be negative')
